In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <08E12B12-6183-307E-BDA0-374FA8EBA2C9> /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/image.so
  Expected in:     <709C1DF5-D253-3C66-87E2-C99FD3A259DF> /Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/ut

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ASPP, self).__init__()
        self.atrous_block1 = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.atrous_block6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=6, dilation=6)
        self.atrous_block12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=12, dilation=12)
        self.atrous_block18 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=18, dilation=18)
        
        self.conv_1x1_output = nn.Conv2d(out_channels * 4, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.atrous_block1(x)
        x2 = self.atrous_block6(x)
        x3 = self.atrous_block12(x)
        x4 = self.atrous_block18(x)
        x = torch.cat([x1, x2, x3, x4], dim=1)
        x = self.conv_1x1_output(x)
        return x

class DeepLabV3Plus(nn.Module):
    def __init__(self, num_classes=1, pretrained_backbone=True, backbone_weights_path=None):
        super(DeepLabV3Plus, self).__init__()
        self.backbone = models.resnet50(pretrained=False)
        if pretrained_backbone and backbone_weights_path:
            state_dict = torch.load(backbone_weights_path)
            self.backbone.load_state_dict(state_dict, strict=False)
        
        # Encoder layers: use layers up to layer4 of ResNet
        self.encoder = nn.Sequential(*list(self.backbone.children())[:-2])

        # ASPP with fixed output
        self.aspp = ASPP(in_channels=2048, out_channels=256)

        # 1x1 Convolution to reduce encoder output channels to 256
        self.encoder_reduction = nn.Conv2d(2048, 256, kernel_size=1)

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, num_classes, kernel_size=1)
        )

    def forward(self, x):
        # Encoder
        encoder_output = self.encoder(x)  # Expected shape: [batch, 2048, h/16, w/16]
        
        # ASPP
        aspp_output = self.aspp(encoder_output)  # Shape: [batch, 256, h/16, w/16]
        
        # Reduce encoder output channels to 256
        encoder_upsampled = self.encoder_reduction(encoder_output)  # Shape: [batch, 256, h/16, w/16]
        
        # Concatenate ASPP output with reduced encoder output
        decoder_input = torch.cat([aspp_output, encoder_upsampled], dim=1)  # Shape: [batch, 512, h/16, w/16]
        
        # Decode
        decoder_output = self.decoder(decoder_input)  # Output shape: [batch, num_classes, h/16, w/16]
        
        # Upsample to match input size
        x = F.interpolate(decoder_output, size=(256, 256), mode='bilinear', align_corners=False)
        return x

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from PIL import Image
from sklearn.metrics import precision_score, recall_score, f1_score, jaccard_score

# 自定义数据集类
class ContrailDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        super().__init__()
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.files = [f for f in self.image_dir.iterdir() if f.is_file() and f.suffix in ['.png', '.jpg', '.jpeg']]
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        image_path = self.files[idx]
        mask_path = self.mask_dir / image_path.name

        image = Image.open(image_path).convert('RGB')  # 读取为RGB图像
        mask = Image.open(mask_path).convert('L')  # 读取为灰度图像

        if self.transform:
            image = self.transform(image)
            mask = transforms.ToTensor()(mask)

        mask = (mask > 0).float()  # 将掩码二值化
        return image, mask.unsqueeze(0)  # 在channel维度增加一个维度

# 定义Dice损失函数
class DiceLoss(nn.Module):
    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
        return 1 - dice

# 计算 Precision, Recall, F1 Score, IoU
def calculate_metrics(preds, labels):
    preds = (torch.sigmoid(preds) > 0.5).cpu().numpy().flatten()  # 二值化预测
    labels = labels.cpu().numpy().flatten()

    precision = precision_score(labels, preds, zero_division=1)
    recall = recall_score(labels, preds, zero_division=1)
    f1 = f1_score(labels, preds, zero_division=1)
    iou = jaccard_score(labels, preds, zero_division=1)

    return precision, recall, f1, iou

# 计算平均 IoU（mIoU），对于二分类问题，mIoU 就是 IoU 本身
def calculate_mIoU(preds, labels):
    preds = (torch.sigmoid(preds) > 0.5).cpu().numpy()
    labels = labels.cpu().numpy()

    ious = []
    for i in range(preds.shape[0]):  # 针对每一张图片
        iou = jaccard_score(labels[i].flatten(), preds[i].flatten(), zero_division=1)
        ious.append(iou)
    
    return sum(ious) / len(ious)

def calculate_accuracy(preds, labels):
    # 使用Sigmoid并将输出二值化
    preds = (torch.sigmoid(preds) > 0.5).float()  # 二值化预测
    # 确保预测和标签维度一致
    preds = preds.view(-1)
    labels = labels.view(-1)

    # 计算正确的预测像素总数
    correct = (preds == labels).sum().item()
    # 计算总像素数
    total = labels.size(0)

    # 返回准确率
    acc = correct / total
    return acc

# 训练函数
def train_model(model, train_loader, valid_loader, loss_fn, optimizer, num_epochs, device):
    best_f1 = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_acc = 0

        # Training phase
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, masks)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            acc = calculate_accuracy(outputs, masks)
            train_acc += acc

        # Validation phase
        model.eval()
        valid_loss = 0
        valid_acc = 0
        valid_precision = 0
        valid_recall = 0
        valid_f1 = 0
        valid_iou = 0
        valid_miou = 0

        with torch.no_grad():
            for images, masks in valid_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)

                loss = loss_fn(outputs, masks)
                valid_loss += loss.item()

                acc = calculate_accuracy(outputs, masks)
                valid_acc += acc

                precision, recall, f1, iou = calculate_metrics(outputs, masks)
                valid_precision += precision
                valid_recall += recall
                valid_f1 += f1
                valid_iou += iou
                valid_miou += calculate_mIoU(outputs, masks)

        # 平均值
        avg_train_loss = train_loss / len(train_loader)
        avg_train_acc = train_acc / len(train_loader)
        avg_valid_loss = valid_loss / len(valid_loader)
        avg_valid_acc = valid_acc / len(valid_loader)
        avg_valid_precision = valid_precision / len(valid_loader)
        avg_valid_recall = valid_recall / len(valid_loader)
        avg_valid_f1 = valid_f1 / len(valid_loader)
        avg_valid_iou = valid_iou / len(valid_loader)
        avg_valid_miou = valid_miou / len(valid_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}, '
              f'Val Loss: {avg_valid_loss:.4f}, Val Acc: {avg_valid_acc:.4f}, '
              f'Precision: {avg_valid_precision:.4f}, Recall: {avg_valid_recall:.4f}, '
              f'F1 Score: {avg_valid_f1:.4f}, IoU: {avg_valid_iou:.4f}, mIoU: {avg_valid_miou:.4f}')

        # 保存最好的模型
        if avg_valid_f1 > best_f1:
            best_f1 = avg_valid_f1
            torch.save(model.state_dict(), 'DeepLabV3+_model.pth')

# Example usage similar to PAN model setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image_dir = '/Users/camus/Desktop/trainX/image'
mask_dir = '/Users/camus/Desktop/trainX/maskimage'

batch_size = 4
num_epochs = 100

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize image
    transforms.ToTensor(),
])

# Load dataset
dataset = ContrailDataset(image_dir, mask_dir, transform=transform)
train_size = int(0.75 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

# Specify backbone weights path
backbone_weights_path = '/Volumes/Vettel/学习/预训练模型/resnet50-0676ba61.pth'

# Instantiate model
model = DeepLabV3Plus(num_classes=1, pretrained_backbone=True, backbone_weights_path=backbone_weights_path).to(device)
loss_fn = DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train model
train_model(model, train_loader, valid_loader, loss_fn, optimizer, num_epochs, device)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/100], Train Loss: 0.5157, Train Acc: 0.8336, Val Loss: 0.5123, Val Acc: 0.8353, Precision: 0.3670, Recall: 0.9600, F1 Score: 0.5183, IoU: 0.3603, mIoU: 0.3819
Epoch [2/100], Train Loss: 0.3513, Train Acc: 0.9293, Val Loss: 0.4728, Val Acc: 0.8705, Precision: 0.4976, Recall: 0.7404, F1 Score: 0.5615, IoU: 0.4072, mIoU: 0.4526
Epoch [3/100], Train Loss: 0.3149, Train Acc: 0.9346, Val Loss: 0.3270, Val Acc: 0.9395, Precision: 0.6941, Recall: 0.7239, F1 Score: 0.6894, IoU: 0.5361, mIoU: 0.5298
Epoch [4/100], Train Loss: 0.3003, Train Acc: 0.9418, Val Loss: 0.3465, Val Acc: 0.9336, Precision: 0.6515, Recall: 0.7122, F1 Score: 0.6637, IoU: 0.5063, mIoU: 0.5271
Epoch [5/100], Train Loss: 0.2873, Train Acc: 0.9428, Val Loss: 0.2974, Val Acc: 0.9387, Precision: 0.6486, Recall: 0.8211, F1 Score: 0.7131, IoU: 0.5679, mIoU: 0.5929
Epoch [6/100], Train Loss: 0.2517, Train Acc: 0.9501, Val Loss: 0.2969, Val Acc: 0.9487, Precision: 0.7723, Recall: 0.6824, F1 Score: 0.7114, IoU: 0.5584, mIoU:

Epoch [50/100], Train Loss: 0.1289, Train Acc: 0.9746, Val Loss: 0.2385, Val Acc: 0.9536, Precision: 0.7684, Recall: 0.7837, F1 Score: 0.7636, IoU: 0.6248, mIoU: 0.6331
Epoch [51/100], Train Loss: 0.1194, Train Acc: 0.9743, Val Loss: 0.2627, Val Acc: 0.9508, Precision: 0.7814, Recall: 0.7206, F1 Score: 0.7388, IoU: 0.5953, mIoU: 0.6072
Epoch [52/100], Train Loss: 0.1138, Train Acc: 0.9777, Val Loss: 0.2411, Val Acc: 0.9536, Precision: 0.7674, Recall: 0.7770, F1 Score: 0.7617, IoU: 0.6236, mIoU: 0.6297
Epoch [53/100], Train Loss: 0.1151, Train Acc: 0.9780, Val Loss: 0.2587, Val Acc: 0.9486, Precision: 0.7286, Recall: 0.7855, F1 Score: 0.7431, IoU: 0.5995, mIoU: 0.6245
Epoch [54/100], Train Loss: 0.1125, Train Acc: 0.9779, Val Loss: 0.2432, Val Acc: 0.9542, Precision: 0.7834, Recall: 0.7563, F1 Score: 0.7602, IoU: 0.6210, mIoU: 0.6216
Epoch [55/100], Train Loss: 0.1064, Train Acc: 0.9792, Val Loss: 0.2318, Val Acc: 0.9551, Precision: 0.7797, Recall: 0.7779, F1 Score: 0.7699, IoU: 0.6340,

Epoch [99/100], Train Loss: 0.0929, Train Acc: 0.9817, Val Loss: 0.2469, Val Acc: 0.9524, Precision: 0.7534, Recall: 0.7815, F1 Score: 0.7551, IoU: 0.6151, mIoU: 0.6330
Epoch [100/100], Train Loss: 0.0913, Train Acc: 0.9820, Val Loss: 0.2457, Val Acc: 0.9530, Precision: 0.7566, Recall: 0.7801, F1 Score: 0.7564, IoU: 0.6163, mIoU: 0.6339
